In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import faiss
from sentence_transformers import SentenceTransformer
from datasets import load_dataset
import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [75]:
dataset = load_dataset("sentence-transformers/quora-duplicates", "pair", split="train[:10000]")

In [76]:
questions = []

for sample in dataset["anchor"]:
    questions.append(sample)

for sample in dataset["positive"]:
    questions.append(sample)

print(questions[:10])
print('-' * 50)
print(f'Number of questions: {len(questions)}')

['Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?', 'How can I be a good geologist?', 'How do I read and find my YouTube comments?', 'What can make Physics easy to learn?', 'What was your first sexual experience like?', 'What would a Trump presidency mean for current international master’s students on an F1 visa?', 'What does manipulation mean?', 'Why are so many Quora users posting questions that are readily answered on Google?', 'Why do rockets look white?', 'How should I prepare for CA final law?']
--------------------------------------------------
Number of questions: 20000


In [77]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [78]:
batch_size = 64
emb_dim = model.get_sentence_embedding_dimension()
index = faiss.IndexFlatIP(emb_dim)

In [79]:
emb = model.encode(questions, normalize_embeddings=True, convert_to_numpy=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [80]:
index.add(emb)

In [81]:
def semantic_search(query, k=5):
    qvec = model.encode(query, normalize_embeddings=True, convert_to_numpy=True)
    scores, ids = index.search(qvec.reshape(1, -1), k)
    for s, idx in zip(scores[0], ids[0]):
        print(f"{s:.3f}  {questions[idx]}")

In [84]:
semantic_search("which country to visit in latin america?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

0.914  What is the best place to visit in Latin America?
0.890  What are the top places to visit in Latin America?
0.653  Which is the best country to visit?
0.637  What are the best countries to visit?
0.637  What are the best countries to visit?
